In [33]:
from sqlalchemy import create_engine
import pandas as pd
from math import radians, sin, cos, sqrt, atan2

In [34]:
# Parámetros de conexión
user = 'root'
password = ''  # Deja la contraseña vacía si no tienes
host = 'localhost'
port = '3306'
database = 'eyc_tarea1'

# Crear el motor de conexión
connection_string = f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}'
engine = create_engine(connection_string)

# Leer datos desde la base de datos
query_edificios = 'SELECT * FROM edificios_escolares'
query_colonias = 'SELECT * FROM cat_colonias'

# Cargar datos en DataFrames
edificios_escolares_df = pd.read_sql(query_edificios, engine)
cat_colonias_df = pd.read_sql(query_colonias, engine)

# Limpiar datos: Reemplazar valores vacíos o no numéricos con NaN
edificios_escolares_df['latitud'] = pd.to_numeric(edificios_escolares_df['latitud'], errors='coerce')
edificios_escolares_df['longitud'] = pd.to_numeric(edificios_escolares_df['longitud'], errors='coerce')

cat_colonias_df['lat'] = pd.to_numeric(cat_colonias_df['lat'], errors='coerce')
cat_colonias_df['lon'] = pd.to_numeric(cat_colonias_df['lon'], errors='coerce')

# Eliminar filas con valores NaN en las columnas relevantes
edificios_escolares_df = edificios_escolares_df.dropna(subset=['latitud', 'longitud'])
cat_colonias_df = cat_colonias_df.dropna(subset=['lat', 'lon'])

# Verificar los tipos de datos
print(edificios_escolares_df.dtypes)
print(cat_colonias_df.dtypes)

cct                                 object
tipo_centro                         object
desc_tipo_centro                    object
turno                               object
desc_turno                          object
servicio                            object
desc_servicio                       object
sostenimiento                       object
desc_sostenimiento                  object
nivel_educativo                     object
desc_nivel_educativo                object
subnivel_educativo                  object
desc_subnivel_educativo             object
nombre                              object
status                              object
desc_status                         object
modalidad                           object
desc_modalidad                      object
fecha_alta                  datetime64[ns]
fecha_baja                  datetime64[ns]
fecha_clausura              datetime64[ns]
fecha_reapertura            datetime64[ns]
fecha_cambio                datetime64[ns]
region     

In [35]:
# Función para calcular la distancia entre dos coordenadas usando la fórmula de Haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en kilómetros

    # Convertir coordenadas de grados a radianes
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # Diferencias de coordenadas
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Aplicar fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

# Unir los dos dataframes en base al código postal
merged_df = pd.merge(edificios_escolares_df, cat_colonias_df, left_on='codigo_postal', right_on='cve_codpost')

# Calcular la distancia entre cada edificio escolar y el centro de su código postal
merged_df['distancia_km'] = merged_df.apply(lambda row: haversine(row['latitud'], row['longitud'], row['lat'], row['lon']), axis=1)

# Filtrar los edificios escolares que están a 10 km o más de distancia
result_df = merged_df[merged_df['distancia_km'] >= 10]

# Mostrar los resultados
result_df_filtered = result_df[['cct', 'nombre', 'codigo_postal', 'distancia_km']]
result_df_filtered

,cct,nombre,codigo_postal,distancia_km
12,05ACC0001O,COORDINACION ESTATAL COAHUILA,00000,11110.880987
21,05ADG0002I,SUBDIRECCION DE SERVICIOS EDUCATIVOS REGION LA...,00000,11353.747730
24,05ADG0004G,DIRECCION DE ENLACE Y VINCULACION CON IES,00000,11106.726341
52,05ADG0008C,PERSONAL COMISIONADO A LA SEPC,00000,11110.880987
56,05ADG0012P,DIRECCION DE SERVICIO COMUNITARIO,00000,11106.726341
...,...,...,...,...
82560,05PCP0005Q,INSTITUTO DIDAXIS DE ESTUDIOS SUPERIORES,00000,11353.747730
82561,05PCP0006P,INSTITUTO ANDRES OSUNA,00000,11353.747730
82562,05PII0002D,INSTITUTO DE IDIOMAS DE TORREON,00000,11353.747730
82572,05PII0004B,BIRMINGHAM INSTITUTE,00000,11353.747730
